In [4]:
a = range(1,5+1)
print (a)

for i in a:
    print(i)

range(1, 6)
1
2
3
4
5


In [1]:
from extractive import ExtractiveSummarizer
import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_checkpoint= "/mnt/data-poseidon/sumire/repos/transformersum/ckpt/epoch=3.ckpt"
model = ExtractiveSummarizer.load_from_checkpoint(model_checkpoint)

#model.to(device)  
text_to_summarize = "And keep it clean!Mr. Chandler!My friends, you are all familiar with Beaugard.This is perhaps his masterpiece."
input_sentences = ["And keep it clean!", "Mr. Chandler!", "My friends, you are all familiar with Beaugard.", "This is perhaps his masterpiece."]
summary = model.predict_sentences(input_sentences, raw_scores=False, num_summary_sentences=1)
print (summary)

ModuleNotFoundError: No module named 'extractive'